# Bag-of-words document classification

What will happen on Reuters?

* How to read the original Reuters data in: [read_news.ipynb](read_news.ipynb)
* Reuters news with about 10000 news articles classified into 66 classes
* We only keep classes with at least 5 examples, end up with 51 classes
* How well can we do on a 51-class classification problem with our BoW?

This is the exact same code as in the original bag-of-words, just file names changed:

In [1]:
import json
import random
random.seed(0)
with open("data/reuters_51cls.json") as f:
    data=json.load(f)
random.shuffle(data) #play it safe!
print(data[0]) #Every item is a dictionary with `text` and `class` keys, here's one:

{'text': '&#2;\nUNITED COMPANIES &lt;UNCF> DECLARES STOCK DIVIDEND\nBATON ROUGE, La, March 6 - United Companies Financial Corp\nsaid its board declared a two pct stock dividend payable APril\neight to holders of record March 17.\nThe board also declared a regular quarterly cash dividend\nof 12.5 cts payable April one to holders of record March 16.\nReuter\n&#3;', 'class': 'earn'}


In [2]:
# We need to gather the texts, into a list
texts=[one_example["text"] for one_example in data]
labels=[one_example["class"] for one_example in data]
print(texts[:2])
print(labels[:2])

['&#2;\nUNITED COMPANIES &lt;UNCF> DECLARES STOCK DIVIDEND\nBATON ROUGE, La, March 6 - United Companies Financial Corp\nsaid its board declared a two pct stock dividend payable APril\neight to holders of record March 17.\nThe board also declared a regular quarterly cash dividend\nof 12.5 cts payable April one to holders of record March 16.\nReuter\n&#3;', '&#2;\nCANBRA FOODS SETS SPECIAL FIVE DLR/SHR PAYOUT\nLETHBRIDGE, Alberta, March 16 - &lt;Canbra Foods Ltd>, earlier\nreporting a 1986 net profit against a year-ago loss, said it\ndeclared a special, one-time dividend of five dlrs per common\nshare, pay March 31, record March 26.\nCanbra said it set the special payout to allow shareholders\nto participate in the gain on the sale of unit Stafford Foods\nLtd in November, 1986, as well as the company\'s "unusually\nprofitable performance" in 1986.\nCanbra earlier reported 1986 net earnings of 4.2 mln dlrs,\nexcluding a 1.3 mln dlr gain on the Stafford sale, compared to\na year-ago loss o

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


vectorizer=CountVectorizer(max_features=100000,binary=True,ngram_range=(1,1)) #unigrams etc., binaryn voi ottaa pois
feature_matrix=vectorizer.fit_transform(texts)
print("shape=",feature_matrix.shape)
#print(feature_matrix.todense())




shape= (9465, 29208)


Now we have the feature matrix done! Next thing we need is the class labels to be predicted in one-hot encoding. This means:

* one row for every example
* one column for every possible class label
* exactly one column has 1 for every example, corresponding to the desired class

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder=LabelEncoder() #Turns class labels into integers
one_hot_encoder=OneHotEncoder(sparse=False) #Turns class integers into one-hot encoding
class_numbers=label_encoder.fit_transform(labels)
print("class_numbers shape=",class_numbers.shape)
print("class_numbers",class_numbers)
print("class labels",label_encoder.classes_)
#And now yet the one-hot encoding
classes_1hot=one_hot_encoder.fit_transform(class_numbers.reshape(-1,1))
print("classes_1hot",classes_1hot)

class_numbers shape= (9465,)
class_numbers [11 11 11 ... 11  0  0]
class labels ['acq' 'alum' 'bop' 'carcass' 'cocoa' 'coffee' 'copper' 'cotton' 'cpi'
 'crude' 'dlr' 'earn' 'fuel' 'gas' 'gnp' 'gold' 'grain' 'heat' 'housing'
 'income' 'instal-debt' 'interest' 'ipi' 'iron-steel' 'jobs' 'lead' 'lei'
 'livestock' 'lumber' 'meal-feed' 'money-fx' 'money-supply' 'nat-gas'
 'oilseed' 'orange' 'pet-chem' 'potato' 'reserves' 'retail' 'rubber'
 'ship' 'silver' 'strategic-metal' 'sugar' 'tea' 'tin' 'trade' 'veg-oil'
 'wpi' 'yen' 'zinc']
classes_1hot [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [5]:
import h5py
from keras.models import Model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint

def save_model(file_name,model,label_encoder,vectorizer):
    """Saves model structure and vocabularies"""
    model_json = model.to_json()
    with open(file_name+".model.json", "w") as f:
        print(model_json,file=f)
    with open(file_name+".vocabularies.json","w") as f:
        classes=list(label_encoder.classes_)
        vocab=dict(((str(w),int(idx)) for w,idx in vectorizer.vocabulary_.items()))
        json.dump((classes,vocab),f,indent=2)
        
example_count,feature_count=feature_matrix.shape
example_count,class_count=classes_1hot.shape

inp=Input(shape=(feature_count,))
hidden=Dense(300,activation="tanh")(inp)
outp=Dense(class_count,activation="softmax")(hidden)
model=Model(inputs=[inp], outputs=[outp])
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy']) #sgd on tyhmin optimizer, vaikuttaa paljon tulokseen!!, "adam" on parempi, mutta voi joskus mennä ihan sekaisin
#googleta ja kokeile eri optimizereja

# Save model and vocabularies
save_model("models/reuters_51cls_bow",model,label_encoder,vectorizer)
# Callback function to save weights during training
save_cb=ModelCheckpoint(filepath="models/reuters_51cls_bow.weights.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
hist=model.fit(feature_matrix,classes_1hot,batch_size=100,verbose=1,epochs=2,validation_split=0.1,callbacks=[save_cb])


/home/marsalv/.local/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 8518 samples, validate on 947 samples
Epoch 1/2
8518/8518 [==============================] - 39s 5ms/step - loss: 0.9820 - acc: 0.8164 - val_loss: 0.4030 - val_acc: 0.9229

Epoch 00001: val_loss improved from inf to 0.40296, saving model to models/reuters_51cls_bow.weights.h5
Epoch 2/2
8518/8518 [==============================] - 38s 4ms/step - loss: 0.1480 - acc: 0.9804 - val_loss: 0.2625 - val_acc: 0.9493

Epoch 00002: val_loss improved from 0.40296 to 0.26253, saving model to models/reuters_51cls_bow.weights.h5


In [ ]:
import numpy
from sklearn.metrics import classification_report, confusion_matrix

#Validation data used during training:
val_instances,val_labels_1hot,_=hist.validation_data

print("Network output=",model.predict(val_instances))#network output should add up to 1 (vrt. 0.8 ja 0.2),
predictions=numpy.argmax(model.predict(val_instances),axis=1)
print("Maximum class for each example=",predictions)#for every row find the max value, rows=validations axis=0, columns=classes axis=1
gold=numpy.nonzero(val_labels_1hot)[1] #undo 1-hot encoding #kertoo missä on ei-nolla arvo (1)
conf_matrix=confusion_matrix(list(gold),list(predictions)) #confusion matrix (predicted,correcrt) 
print(conf_matrix)
print(classification_report(list(gold),list(predictions),target_names=label_encoder.classes_))
#support=kuinka monta kpl olisi pitäinyt löytää
#recall=kuinka monta löysi, niistä mitä piti
#precision=?
#f=precision ja recall yhteensä
